In [1]:
%run ../DataClass.ipynb

In [2]:
X_train = np.load('./0/data/X_train.npy')
X_test = np.load('./0/data/X_test.npy')
X_val = np.load('./0/data/X_val.npy')
y_train = np.load('./0/data/y_train.npy')
y_test = np.load('./0/data/y_test.npy')
y_val = np.load('./0/data/y_val.npy')

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
from pprint import pprint

# Tuning RF

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 200, num = 5)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(1, 45, num = 3)]
# Minimum number of samples required to split a node
min_samples_split = [5, 10]


# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split}

print(random_grid)

In [ ]:
%%time
rf_random = RandomizedSearchCV(estimator = RandomForestClassifier(), param_distributions = random_grid, n_iter = 10, cv = 5, verbose=2)
# Fit the random search model
rf_random.fit(X_train, y_train)

In [ ]:
rf_random.best_estimator_ , rf_random.best_score_

In [ ]:
type(rf_random.best_estimator_)

In [12]:
# RF = rf_random.best_estimator_
RF = RandomForestClassifier(max_depth=45,n_estimators=300,random_state=0)

In [ ]:
%%time
RF.fit(X_train,y_train)
y_pred_val = RF.predict(X_val)
y_pred_test = RF.predict(X_test)

print('val accuracy:',accuracy_score(y_val, y_pred_val))
print('test accuracy:',accuracy_score(y_test, y_pred_test))

# GB tuning

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 70, num = 4)]
learning_rate = [x for x in np.linspace(start = 0.05, stop = 0.2, num = 4)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(start = 5, stop = 16, num = 4)]


# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'learning_rate' : learning_rate,
               'max_features': max_features,
               'max_depth': max_depth}

pprint(random_grid)

In [ ]:
%%time
GBDT_random = RandomizedSearchCV(estimator = GradientBoostingClassifier(), param_distributions = random_grid, n_iter = 10, cv = 5, verbose=2)
# Fit the random search model
GBDT_random.fit(X_train, y_train)

In [ ]:
GBDT_random.best_estimator_ , GBDT_random.best_score_

In [ ]:
%%time
# GB = rf_random.best_estimator_
GB = GradientBoostingClassifier(learning_rate=0.2, max_depth=8, max_features='auto',
                            n_estimators=20, random_state=0)
GB.fit(X_train,y_train)
y_pred_val = GB.predict(X_val)
y_pred_test = GB.predict(X_test)

print('val accuracy:',accuracy_score(y_val, y_pred_val))
print('test accuracy:',accuracy_score(y_test, y_pred_test))

In [57]:
if not os.path.exists('./model_params/'):
    os.makedirs('./model_params')

with open('./model_params/GB_params_tuning.pkl', 'wb') as f:
        pickle.dump(GBDT_random.best_estimator_, f)
with open('./model_params/RF_params_tuning.pkl', 'wb') as f:
        pickle.dump(rf_random.best_estimator_, f)

# 1-NN

In [5]:
OneNN_model = KNeighborsClassifier(n_neighbors=1).fit(X_train, y_train)

In [7]:
y_pred_val = OneNN_model.predict(X_val)
y_pred_test = OneNN_model.predict(X_test)
print('val accuracy:',accuracy_score(y_val, y_pred_val))
print('test accuracy:',accuracy_score(y_test, y_pred_test))

val accuracy: 0.9695714285714285
test accuracy: 0.9677142857142857


In [8]:
if not os.path.exists('./model_params/'):
    os.makedirs('./model_params')
with open('./model_params/OneNN_params_tuning.pkl', 'wb') as f:
        pickle.dump(OneNN_model, f)

# CNN

In [54]:
from tensorflow.keras.optimizers import SGD
pixels  = int(sqrt(X_train.shape[1]))
classes = len(np.unique(y_test))

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(10, activation='softmax'))
# compile model
opt = SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.save(f'./model_params/CNN_params_tuning')

In [61]:
#load model
num_labels = classes
pixels = int(sqrt(X_train.shape[1]))
# Reshaping to format which CNN expects (batch, height, width, channels)
trainX_cnn = X_train.reshape(X_train.shape[0], pixels, pixels, 1).astype('float32')
valX_cnn = X_val.reshape(X_val.shape[0], pixels, pixels, 1).astype('float32')
testX_cnn= X_test.reshape(X_test.shape[0], pixels, pixels, 1).astype('float32')

# Normalize images from 0-255 to 0-1
trainX_cnn /= 255
valX_cnn /= 255

train_y_cnn = utils.to_categorical(y_train, num_labels)
val_y_cnn = utils.to_categorical(y_val, num_labels)
test_y_cnn = utils.to_categorical(y_test, num_labels)
# Learn model
model.fit(trainX_cnn, train_y_cnn, validation_data=(valX_cnn, val_y_cnn), epochs=10, batch_size=32)



Epoch 1/10
1313/1313 [==============================] - 10s 7ms/step - loss: 0.2203 - accuracy: 0.9335 - val_loss: 0.0968 - val_accuracy: 0.9711
Epoch 2/10
1313/1313 [==============================] - 10s 8ms/step - loss: 0.0728 - accuracy: 0.9787 - val_loss: 0.0629 - val_accuracy: 0.9812
Epoch 3/10
1313/1313 [==============================] - 10s 7ms/step - loss: 0.0440 - accuracy: 0.9864 - val_loss: 0.0517 - val_accuracy: 0.9839
Epoch 4/10
1313/1313 [==============================] - 9s 7ms/step - loss: 0.0294 - accuracy: 0.9916 - val_loss: 0.0558 - val_accuracy: 0.9831
Epoch 5/10
1313/1313 [==============================] - 10s 7ms/step - loss: 0.0199 - accuracy: 0.9940 - val_loss: 0.0524 - val_accuracy: 0.9848
Epoch 6/10
1313/1313 [==============================] - 9s 7ms/step - loss: 0.0138 - accuracy: 0.9957 - val_loss: 0.0499 - val_accuracy: 0.9854
Epoch 7/10
1313/1313 [==============================] - 9s 7ms/step - loss: 0.0083 - accuracy: 0.9981 - val_loss: 0.0523 - val_accur

In [62]:
y_pred_val=np.argmax(model.predict(valX_cnn),axis=1)
y_pred_test=np.argmax(model.predict(testX_cnn),axis=1)

In [63]:
print('val accuracy:',accuracy_score(y_val, y_pred_val))
print('test accuracy:',accuracy_score(y_test, y_pred_test))

val accuracy: 0.9860714285714286
test accuracy: 0.9707857142857143
